In [1]:
import os, sys, numpy as np
os.environ['SPARK_HOME']="/Users/paulthompson/spark-1.6.1-bin-hadoop2.4"
sys.path.append("/Users/paulthompson/spark-1.6.1-bin-hadoop2.4/python/")
from pyspark import SparkConf, SparkContext
conf = (SparkConf().setMaster("local").setAppName("My app").set("spark.executor.memory", "1g"))
from pyspark.sql import SQLContext
from pyspark.sql.dataframe import StructType, StructField, IntegerType, FloatType
sc = SparkContext(conf = conf)
sqlContext = SQLContext(sc)

In [2]:
mv_ratings = sc.textFile('/Users/paulthompson/Documents/MSAN_Files/Spr2_Distributed/HW1/movies/ratings.txt')

In [3]:
user_item_ratings = mv_ratings.map(lambda line: (line.split(':')[0], (line.split(':')[1], line.split(':')[2])))
user_item_ratings = user_item_ratings.groupByKey()

In [4]:
item_user_ratings = mv_ratings.map(lambda line: (line.split(':')[1], (line.split(':')[0], line.split(':')[2])))
item_user_ratings = item_user_ratings.groupByKey()

In [15]:
lambda_ = 0.1; n_factors = 3; n_iterations = 20

In [6]:
Users = user_item_ratings.map(lambda line: (line[0][0], 5 * np.random.rand(1, n_factors)))
Items = item_user_ratings.map(lambda line: (line[0][0], 5 * np.random.rand(1, n_factors)))

In [7]:
print Items.take(10)

[(u'6', array([[ 1.76099363,  0.44535205,  2.34266234]])), (u'4', array([[ 1.10675171,  2.00015634,  4.7637379 ]])), (u'5', array([[ 1.88378986,  2.29429409,  4.53671826]])), (u'2', array([[ 3.01522963,  3.65414647,  4.30972223]])), (u'5', array([[ 4.18950143,  0.81001435,  3.89523736]])), (u'3', array([[ 2.55368139,  2.24010961,  3.91618921]])), (u'3', array([[ 0.24647697,  1.94724627,  2.0248009 ]])), (u'3', array([[ 2.30765205,  0.88326048,  1.1351706 ]])), (u'3', array([[ 3.35541701,  0.14301716,  1.06858676]])), (u'3', array([[ 4.85158368,  1.27097163,  4.74590465]]))]


In [8]:
Items_broadcast = sc.broadcast({
  k: v for (k, v) in Items.collect()
})

In [10]:
j = 0
for k, v in {k: v for (k, v) in Items.collect()}.iteritems():
    print k, v
    j+=1
    if j > 10:
        break

1 [[ 1.39407107  0.01353967  1.79493675]]
3 [[ 1.65094866  1.85772861  2.68837578]]
2 [[ 1.18634226  0.22802199  2.38712183]]
5 [[ 4.20489393  4.89298562  2.87135711]]
4 [[ 2.6610737   3.17496804  1.60279449]]
7 [[ 0.53475988  0.68879375  4.72762786]]
6 [[ 2.34837859  3.28342059  2.97283678]]
9 [[ 3.05060587  1.92404819  1.22090204]]
8 [[ 0.48339076  2.4826844   2.72024888]]


In [16]:
joint_U_A = Users.join(user_item_ratings)
print joint_U_A.take(2)

[(u'5', (array([[ 2.30765205,  0.88326048,  1.1351706 ]]), <pyspark.resultiterable.ResultIterable object at 0x10439cfd0>)), (u'5', (array([[ 3.35541701,  0.14301716,  1.06858676]]), <pyspark.resultiterable.ResultIterable object at 0x10439c750>))]


In [14]:
def Update_U_A(jointTuple, items_broadcast):
    Itemssquare = np.zeros([3,3])
    for k, v in {k: v for (k, v) in Items.collect()}.iteritems():
        for i in Itemssquare.shape[0]:
            for j in Itemssquare.shape[1]:
                Itemssquare[i,j] += v[0][i]*v[0][j]
    
    
    
    
    
    
    
    
    

In [ ]:
# Users = joint_U_A.map(Update_U_A())